In [1]:
import numpy as np
from collections import Counter
from textblob import TextBlob
from nltk.corpus import inaugural
from collections import defaultdict
from IPython.core.display import HTML

In [34]:
from keras.layers.embeddings import Embedding
from keras.utils.generic_utils import Progbar
from keras.utils import np_utils

In [3]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.datasets.data_utils import get_file
import numpy as np
import random
import sys

In [4]:
num_words = len(inaugural.words())
vocab_size = len(set(inaugural.words()))
embed_size = 50

In [5]:
word_to_index = dict(zip(set(inaugural.words()), range(vocab_size)))
inaugural_indices = map(lambda w: word_to_index[w], inaugural.words())

In [6]:
x = 'abcdefghijk'
x[3-8:3] + x[4:4+2]

'ef'

In [7]:
for a, b in enumerate('abcde'):
    print(a)

0
1
2
3
4


In [8]:
32 * (len(inaugural.words())) * vocab_size

45487974080L

In [9]:
np.memmap("test", mode="w+", shape=(4,9), dtype=np.int8)

memmap([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [10]:
model = Sequential()
model.add(Dense(input_dim=vocab_size, output_dim=embed_size, init="glorot_uniform", activation="relu"))
# model.add(Activation("relu"))
model.add(Dense(input_dim=embed_size, output_dim=vocab_size, init="glorot_uniform", activation="softmax"))
# model.add(Activation("softmax"))

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='sgd')

In [21]:
progbar = Progbar(len(inaugural.sents()))
for i, sent in enumerate(inaugural.sents()):
    count = 0
    X = np.zeros((len(sent), vocab_size), dtype=np.int8)
    y = np.zeros((len(sent), vocab_size))
    for j, word in enumerate(sent):
        context_words = sent[j-3:j] + sent[j:j+3]
        for context_word in context_words:
            X[count, word_to_index[context_word]] = 1
        y[count, word_to_index[word]] = 1
        count += 1
    model.train_on_batch(X, y)
    progbar.add(1)

4868/4868 [==============================] - 930s   


In [23]:
json_string = model.to_json()
open('my_model_architecture.json', 'w').write(json_string)
model.save_weights('my_model_weights.h5')

In [195]:
l1 = model.layers[1]
weights = l1.get_weights()[0]
weights = np_utils.normalize(weights)

In [204]:
print(model.layers[0].get_weights()[0].shape)
print(model.layers[1].get_weights()[0].shape)

(9754, 50)
(50, 9754)


In [44]:
weights

array([[-0.00967298, -0.00405135, -0.0049775 , ..., -0.00206025,
         0.01555367,  0.00406158],
       [ 0.00518796,  0.01792908, -0.01338787, ..., -0.00090379,
         0.00691495,  0.01170139],
       [-0.00017346,  0.00758486,  0.00169428, ..., -0.00895134,
        -0.00212953, -0.00612701],
       ..., 
       [-0.00457837, -0.00442814,  0.00974255, ...,  0.00301745,
        -0.00962469, -0.00936329],
       [ 0.00915287,  0.0166948 ,  0.00241083, ...,  0.00226915,
         0.00067894,  0.00652266],
       [ 0.01610753,  0.01233972, -0.0084404 , ...,  0.01396418,
        -0.00886008, -0.0142704 ]])

In [193]:
word_to_vec = dict(map(lambda w: (w, weights.T[word_to_index[w]]), word_to_index.keys()))

In [50]:
Counter(inaugural.words()).most_common()

[(u'the', 9281),
 (u'of', 6970),
 (u',', 6840),
 (u'and', 4991),
 (u'.', 4676),
 (u'to', 4311),
 (u'in', 2527),
 (u'a', 2134),
 (u'our', 1905),
 (u'that', 1688),
 (u'be', 1460),
 (u'is', 1403),
 (u'we', 1141),
 (u'for', 1075),
 (u'by', 1036),
 (u'it', 1011),
 (u'which', 1002),
 (u'have', 994),
 (u'not', 916),
 (u'as', 888),
 (u'with', 886),
 (u'will', 846),
 (u'I', 831),
 (u'are', 774),
 (u'all', 758),
 (u'their', 719),
 (u'this', 700),
 (u'The', 619),
 (u'has', 611),
 (u'people', 559),
 (u'its', 554),
 (u';', 544),
 (u'or', 537),
 (u'from', 521),
 (u'on', 496),
 (u'We', 483),
 (u'been', 482),
 (u'but', 479),
 (u'can', 457),
 (u'us', 455),
 (u'my', 449),
 (u'no', 406),
 (u'an', 377),
 (u'upon', 363),
 (u'--', 363),
 (u'It', 356),
 (u'who', 356),
 (u'so', 354),
 (u'must', 345),
 (u'they', 341),
 (u'Government', 331),
 (u'them', 327),
 (u'States', 325),
 (u'should', 318),
 (u'at', 316),
 (u'may', 316),
 (u'world', 312),
 (u'great', 312),
 (u'shall', 310),
 (u'more', 302),
 (u'country', 2

In [76]:
a = np.array([1,2,3])
np.dot(a, a)

14

In [77]:
from Queue import PriorityQueue
from scipy.spatial.distance import cosine

In [194]:
p = word_to_vec['communism']
q = PriorityQueue()
for word, vec in word_to_vec.iteritems():
    q.put((cosine(p, vec), word))
    
for i in range(10):
    print(q.get())

(0.0, u'communism')
(0.4751598691412916, u'folly')
(0.48087401642689465, u'precepts')
(0.51867168507851946, u'antifederal')
(0.54855645247615359, u'using')
(0.54993997470745182, u'counteraction')
(0.55947034806088636, u'soon')
(0.55996618325643155, u'skies')
(0.5637228822959548, u'vain')
(0.56419072459813924, u'praises')


In [ ]:
inaugural_indices

In [ ]:
'''
    Example script to generate text from Nietzsche's writings.

    At least 20 epochs are required before the generated text
    starts sounding coherent.

    It is recommended to run this script on GPU, as recurrent
    networks are quite computationally intensive.

    If you try this script on new data, make sure your corpus
    has at least ~100k characters. ~1M is better.
'''

In [ ]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

In [ ]:
chars = set(text)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

In [ ]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
y.shape

In [ ]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

In [ ]:
sample([1,2,3,4,5,6,7])

In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for iteration in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()